Notebook for testing if gdal installaiton works
Code modified from here: https://notebook.community/Automating-GIS-processes/Lesson-7-Automating-Raster-Data-Processing/Python-and-Gdal

In [ ]:
!pip install matplotlib
!pip install seaborn
!pip install rioxarray
!pip install earthpy
!pip install simplekml 
!pip install gdal

In [12]:
import os
import asdc
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import rioxarray as rxr
import earthpy as et
from osgeo import gdal

# Prettier plotting with seaborn
sns.set(font_scale=1.5, style="whitegrid")
!pwd

/home/jovyan/tim.brown@anu.edu.au/test-gdal


Select the task you want to access the raster from

In [88]:
print("Select the task to download the DSM and DTM from. Make sure the Task you have selected actually has both a DSM and a DTM file")
asdc.task_select()
from ipywidgets import widgets

Select the task to download the DSM and DTM from. Make sure the Task you have selected actually has both a DSM and a DTM file


interactive(children=(Dropdown(description='project', index=4, options=(('8: Testing dev platform', 8), ('11: …

interactive(children=(Dropdown(description='task', options=(('Chain Of Lagoons - 30/03/2018 - Resized 2048', '…

Button(description='Run all below', icon='play', style=ButtonStyle())

In [89]:
project_id, task_id = asdc.get_selection()
task_name = asdc.task_dict[task_id]['name']

print("Task name = ", task_name)

Task name =  Chain Of Lagoons - 30/03/2018


In [90]:
project_name = asdc.project_dict[f"{project_id}"]['name']

print(project_name)
task_id

Chain of Lagoons


'2af50c4e-4418-4f34-8535-3518c7f39154'

In [94]:
asdc.download_asset(f"odm_orthophoto/odm_orthophoto.tif", "orthoph.tif")
#DTM_path = f"https://asdc.cloud.edu.au/api/projects/{project_id}/tasks/{task_id}/download/dtm.tif"
print(project_id)
print(task_id)

  0%|          | 0.00/230M [00:00<?, ?iB/s]

25
2af50c4e-4418-4f34-8535-3518c7f39154


In [69]:
# List available TIF files in the selected project
# For files that have multiple copies (eg jpg or laz), provide the file count:


asdc_root_url = 'https://dev.asdc.cloud.edu.au/api/projects/' # Remove .dev if on production

u = asdc.call_api(f"{asdc_root_url}{project_id}/tasks/{task_id}/assets/files.json").json()
filename = ""
print(f"Project name: {project_name}")
print(f"Task Name: {task_name}\n\n")
print("Files in Task:")


from collections import Counter
import os

# List of file extensions to filter by
filter_extensions = ['.tif']

# Get file extensions and count them
file_extensions = [os.path.splitext(file)[1] for file in u['files'] if os.path.splitext(file)[1] in filter_extensions]
file_counts = Counter(file_extensions)

# Separate file types with multiple files and those with a single file
single_files = []
multiple_files = []

for ext, count in file_counts.items():
    # Remove the leading dot from the extension
    ext = ext[1:]
    if count > 1:
        multiple_files.append((ext, count))
    else:
        # Find the full filename for this extension
        for file in u['files']:
            if file.endswith('.' + ext):
                single_files.append(file)
                break

# Print single files first
for file in single_files:
    print(f"  * {file}")

# Then print file types with multiple files
for ext, count in multiple_files:
    print(f"  * {ext.upper()} ({count})")



Project name: USLN
Task Name: BVZ-USLN-COOLAMA-2022-02-11-P4M-TB-RGB


Files in Task:
  * assets/odm_orthophoto/odm_orthophoto.tif


Now download the TIF you want to process. 

In [71]:
base_name_for_outputs =  task_name.split(" |")[0]
temp_folder_name = "TEMP-" + base_name_for_outputs

task_data_folder_path = os.path.join(".", temp_folder_name)  # The '.' represents the current directory

print("Temporary files will be stored in the folder: \n\t" + task_data_folder_path)
print("\nNew files with have the base file name: \n\t" + base_name_for_outputs + "\n-----------------------------")

if not os.path.exists(task_data_folder_path):
    os.makedirs(task_data_folder_path)

Temporary files will be stored in the folder: 
	./TEMP-BVZ-USLN-COOLAMA-2022-02-11-P4M-TB-RGB

New files with have the base file name: 
	BVZ-USLN-COOLAMA-2022-02-11-P4M-TB-RGB
-----------------------------


In [108]:
# Use this code to download via URL rather than with the ASDC API
#  or if you want to have the DSM/DTM files have the task name
#   DSM_path = f"https://asdc.cloud.edu.au/api/projects/{project_id}/tasks/{task_id}/download/dsm.tif"
#DTM_path = f"https://asdc.cloud.edu.au/api/projects/{project_id}/tasks/{task_id}/download/dtm.tif"

# Note that the "download_assets" api call takes the filename (on ODM) as the first variable and the "path/filename" for it to be downloaded to as the second variable
# Also note that unlike the DSM and DTM's, the orthophoto is in a sub-folder


#NOTE: In ODM, the DEM is called a DTM

## 2023-12-07- This code was copied from the tree segmentation code... currently is broken but should be adapted to let the user download all tifs as needed
#print("Downloading DSM and DTM files to: ", task_data_folder_path)
# dtm_filename = 'dtm.tif'
# dsm_filename = 'dsm.tif'
# ortho_filename = 'odm_orthophoto/odm_orthophoto.tif'

# #This is the path to where the downlaoded file will be saved in Jupyter
# dtm_path = task_data_folder_path + "/" + dtm_filename
# dsm_path = task_data_folder_path + "/" + dsm_filename
# ortho_path = task_data_folder_path + ortho_filename
# print(ortho_filename)

# asdc.download_asset(dsm_filename, dsm_path)
# asdc.download_asset(dtm_filename, dtm_path)

downloaded_tif_name = "ortho.tif"
print('Now downloading available raster files. You will see a <404> error for any files that are not in the project')
asdc.download_asset(f"odm_orthophoto/odm_orthophoto.tif", downloaded_tif_name)



Now downloading available raster files. You will see a <404> error for any files that are not in the project


'ortho.tif'

In [109]:
# For now this is hardcoded for the "odm_orthophoto.tif"
# Code should be updated to let the user chose which orthophotos they want



# Open the file:
raster = gdal.Open(downloaded_tif_name)

# Check type of the variable 'raster'
type(raster)

osgeo.gdal.Dataset

In [110]:
# Projection
raster.GetProjection()

# Dimensions
raster.RasterXSize
raster.RasterYSize

# Number of bands
raster.RasterCount

# Metadata for the raster dataset
raster.GetMetadata()

{'AREA_OR_POINT': 'Area'}

In [111]:
# Read the raster band as separate variable
band = raster.GetRasterBand(1)

# Check type of the variable 'band'
type(band)

# Data type of the values
gdal.GetDataTypeName(band.DataType)

'Byte'

In [112]:
# Compute statistics if needed
if band.GetMinimum() is None or band.GetMaximum()is None:
    band.ComputeStatistics(0)
    print("Statistics computed.")
    
# Fetch metadata for the band
band.GetMetadata()
    
# Print only selected metadata:
print ("[ NO DATA VALUE ] = ", band.GetNoDataValue()) # none
print ("[ MIN ] = ", band.GetMinimum())
print ("[ MAX ] = ", band.GetMaximum())

Statistics computed.
[ NO DATA VALUE ] =  None
[ MIN ] =  0.0
[ MAX ] =  255.0
